import cv2
import numpy as np
import torch
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import time

# Load model YOLOv8 yang sudah dilatih untuk deteksi cabai

yolo_model = YOLO('path/to/your/yolov8_chili_model.pt')

# Load model CNN untuk klasifikasi kondisi cabai (busuk atau tidak)

cnn_model = load_model('path/to/your/cnn_chili_condition_classifier.h5')

# Konfigurasi ukuran input untuk model CNN

CNN_INPUT_SIZE = (224, 224) # Sesuaikan dengan ukuran input model CNN Anda

# Fungsi untuk memproses frame video

def process_frame(frame): # Deteksi cabai dengan YOLOv8
results = yolo_model(frame)

    # Untuk setiap hasil deteksi
    for result in results:
        boxes = result.boxes.cpu().numpy()

        for box in boxes:
            # Ekstrak koordinat bbox
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Potong gambar cabai dari frame
            chili_crop = frame[y1:y2, x1:x2]

            if chili_crop.size == 0:
                continue

            # Preprocess untuk model CNN
            chili_resized = cv2.resize(chili_crop, CNN_INPUT_SIZE)
            chili_normalized = chili_resized / 255.0
            chili_input = np.expand_dims(chili_normalized, axis=0)

            # Prediksi kondisi cabai dengan model CNN
            prediction = cnn_model.predict(chili_input)
            condition = "Busuk" if prediction[0][0] > 0.5 else "Baik"
            confidence = prediction[0][0] if condition == "Busuk" else 1 - prediction[0][0]

            # Visualisasi hasil
            color = (0, 0, 255) if condition == "Busuk" else (0, 255, 0)  # Merah untuk busuk, hijau untuk baik
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{condition} ({confidence:.2f})", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return frame

# Fungsi untuk memproses video

def process_video(video_path, output_path=None):
cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Tidak dapat membuka video.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Siapkan video output jika path disediakan
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Proses frame
        processed_frame = process_frame(frame)

        # Simpan ke video output jika diperlukan
        if output_path:
            out.write(processed_frame)

        # Tampilkan frame (opsional, untuk debugging)
        cv2.imshow('Deteksi dan Klasifikasi Cabai', processed_frame)

        frame_count += 1

        # Keluar jika tombol 'q' ditekan
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Hitung dan tampilkan FPS
    elapsed_time = time.time() - start_time
    avg_fps = frame_count / elapsed_time
    print(f"Average FPS: {avg_fps:.2f}")

    # Bersihkan
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

# Contoh penggunaan

if **name** == "**main**": # Untuk kamera webcam # process_video(0, "output_webcam.mp4")

    # Untuk file video
    process_video("path/to/input/video.mp4", "path/to/output/video.mp4")
